In [1]:
# Import libraries.
import numpy as np
import pandas as pd
import spacy
import string

from joblib import dump, load

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, mean_absolute_error, precision_score, recall_score
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Load data.
df = pd.read_csv('../data/first_week_oct_2015_comments_by_top_400_with_scores_and_features_v2.csv')

In [3]:
# Preview data.
df.head()

,id,parent,by,time,hour_posted,text,dead,ranking,text_len,pct_caps,tags_per_char,papi_toxicity,v_neg,v_neu,v_pos,v_compound,tb_polarity,tb_subjectivity,tb_nb_prob_neg,pc_prob_offensive
0,10331981,10331895,debacle,2015-10-05 14:24:42+00:00,14,US is not really scared by BRICS at all. They'...,False,0,146,0.068493,0.000000,0.100881,0.000,0.744,0.256,0.7859,0.156250,0.61875,0.080001,0.238871
1,10343811,10343761,sarciszewski,2015-10-07 02:13:15+00:00,2,"I wasn't really trying to argue, they said the...",False,0,76,0.013158,0.000000,0.048637,0.195,0.805,0.000,-0.3947,0.200000,0.20000,0.593498,0.050161
2,10331538,10331008,debacle,2015-10-05 13:08:10+00:00,13,The examples on the homepage kind of underscor...,False,12,88,0.034091,0.000000,0.044777,0.000,0.864,0.136,0.2975,0.137500,0.50000,0.336201,0.098511
3,10340097,10339965,debacle,2015-10-06 16:33:06+00:00,16,No mention of a critical aspect of a service l...,False,22,99,0.010101,0.000000,0.035335,0.214,0.667,0.119,-0.2500,0.000000,0.80000,0.352414,0.056323
4,10338552,10337763,debacle,2015-10-06 13:06:26+00:00,13,I think some of these points are gross exagger...,False,38,868,0.013825,0.009217,0.232577,0.081,0.902,0.017,-0.8233,0.076667,0.26000,0.000790,0.001499


In [4]:
# Review column list.
df.columns

Index(['id', 'parent', 'by', 'time', 'hour_posted', 'text', 'dead', 'ranking',
       'text_len', 'pct_caps', 'tags_per_char', 'papi_toxicity', 'v_neg',
       'v_neu', 'v_pos', 'v_compound', 'tb_polarity', 'tb_subjectivity',
       'tb_nb_prob_neg', 'pc_prob_offensive'],
      dtype='object')

In [5]:
# Define target column.
y = df['papi_toxicity']

## Text processing with spacy

In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
# Split data into train and test subsets.
X_train, X_test, y_train, y_test = train_test_split(df['text'], 
                                                    y, 
                                                    train_size=0.8, 
                                                    random_state=0)

In [8]:
# Convert comment text to lowercase, remove punctuation, and trim 
# whitespace.
simple_text_train = X_train.str.lower()
simple_text_train = simple_text_train.str.replace('[{}]'.format(string.punctuation), '')
simple_text_train = simple_text_train.str.replace('\s+', ' ', regex=True)
simple_text_train = simple_text_train.str.strip()

simple_text_test = X_test.str.lower()
simple_text_test = simple_text_test.str.replace('[{}]'.format(string.punctuation), '')
simple_text_test = simple_text_test.str.replace('\s+', ' ', regex=True)
simple_text_test = simple_text_test.str.strip()

In [9]:
# Check results.
for text in simple_text_train.head():
    print(text, '\n')

without the safe harbor agreement you can no longer avoid eu privacy regulations by storing the data in the us maybe im missing something here my understanding is that the safe harbour agreement wasnt a mechanism for us companies to avoid eu data protection regulations it was a certification that they did comply with eu data protection particularly in situations where that data was transmitted outside the eu now its gone eu customer data held by us companies will be governed by national data protection laws instead so may end up having to be stored within the eu the us privacy regulations are no longer considered compatible with the eu privacy regulations i dont think they ever were which is why the safe harbour needed to exist in the first place 

imagine how much a interlispd smalltalk mesacedar workstation would have cost in the 70s versus a plain pdp11 eh if theyd put altos in serial production instead of small batches at a time adding up to 2000 units it wouldnt have been vastly m

In [10]:
# Tokenize text.
tokens_train = simple_text_train.apply(nlp.tokenizer)
tokens_test = simple_text_test.apply(nlp.tokenizer)

In [11]:
# # Lemmatize; remove stop words.
# lemmas_train = tokens_train.apply(lambda x: [token.lemma_ for token in x if not token.is_stop])
# lemmas_test = tokens_test.apply(lambda x: [token.lemma_ for token in x if not token.is_stop])

In [12]:
# Lemmatize; leave stop words. Will feature selection pick up "you"?
lemmas_train = tokens_train.apply(lambda x: [token.lemma_ for token in x])
lemmas_test = tokens_test.apply(lambda x: [token.lemma_ for token in x])

In [13]:
# Check results.
lemmas_train.head()

1362    [without, the, safe, harbor, agreement, you, c...
7304    [imagine, how, much, a, interlispd, smalltalk,...
4237    [thank, for, the, link, a, the, current, defac...
3680             [surely, you, can, give, a, rough, idea]
4780    [disclaimer, -PRON-, be, a, browser, engine, d...
Name: text, dtype: object

In [14]:
# Instantiate vectorizer object.
tfidf = TfidfVectorizer(min_df=0.0001, max_df=0.9999)

# Create a vocabulary and get word counts per document.
dtm_train = tfidf.fit_transform(lemmas_train.astype(str))
dtm_test = tfidf.transform(lemmas_test.astype(str))

# Get feature names to use as dataframe column headers.
dtm_train_orig = pd.DataFrame(dtm_train.todense(), columns=tfidf.get_feature_names())
dtm_test_orig = pd.DataFrame(dtm_test.todense(), columns=tfidf.get_feature_names())

# Copy document term matrices.
dtm_train = dtm_train_orig.copy()
dtm_test = dtm_test_orig.copy()

# Preview feature matrix.
dtm_train.head()

,000,0000,00000,0000005,00001,0000ff,0001,000158,00016w,0003,...,никогда,но,правда,развития,скажем,событий,так,тем,уехать,ಠಠ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
dump(tfidf, 'tfidf.joblib')

['tfidf.joblib']

## Manual feature (word) selection

In [16]:
# Convert PerspectiveAPI toxicity scores to binary class labels.
y_train_binary = (y_train > 0.7).astype(int)
y_test_binary = (y_test > 0.7).astype(int)

In [17]:
# Check target distribution.
np.bincount(y_train_binary)

array([7893,   83], dtype=int64)

In [18]:
wordlist = ['addict', 
            'agenda', 
            'alllivesmatter',
            'anal', 
            'analretentive', 
            'anticorporate',
            'antihomosexuality', 
            'antivaxxer', 
            'antivaxxers', 
            'arse', 
            'arsehole',
            'asshole',
            'assholes',
            'awesome',
            'awesomeness',
            'banal',
            'banality',
            'basement',
            'batshit',
            'bitch',
            'blacklivesmatter',
            'bloody',
            'blow',
            'blowjobs',
            'blowup',
            'boob',
            'brain',
            'braindead',
            'brainfart',
            'brainfuck',
            'brilliant', 
            'brilliantly',
            'bro',
            'bros',
            'broscience',
            'broke',
            'brokenness',
            'brothel',
            'bullshit',
            'bullshitfree',
            'bum',
            'bummer',
            'cargocult',
            'celebrity',
            'cheerleader',
            'circlejerk',
            'clown',
            'clusterfuck',
            'clusterfucks',
            'cock',
            'cockamamie',
            'comment',
            'communist',
            'corporate',
            'cowardly',
            'cowboy',
            'crap',
            'crappy',
            'crapshoot',
            'craptastic',
            'crapware',
            'crazy',
            'cryptofascist',
            'cuckold',
            'cuckoo',
            'cult',
            'damn',
            'democrat',
            'deserve',
            'dick',
            'dicked',
            'dickhead',
            'dicking',
            'dicks',
            'dildo',
            'dumb',
            'dumbly',
            'elite',
            'excrement',
            'fag',
            'fail',
            'failing',
            'fake', 
            'fakery',
            'false', 
            'falsely',
            'fart',
            'fascist',
            'feel',
            'fool',
            'foolish',
            'foolproof',
            'forcedvaxxers',
            'fox',
            'fuck',
            'fuckedupness',
            'fullblown',
            'garbage',
            'genius',
            'goddamn',
            'goddamned',
            'greed',
            'greedy',
            'gross',
            'harebrained',
            'hell',
            'hellish',
            'helluva',
            'heterosexual',
            'hitler',
            'hitlers',
            'homo',
            'homophobia',
            'homophobic',
            'homosexual',
            'hooker',
            'idiot',
            'idiotic',
            'injustice',
            'irrational', 
            'irrationally',
            'jerk',
            'justice',
            'kneejerk',
            'knob',
            'leader',
            'lesbian',
            'liar',
            'lol',
            'loser',
            'maga',
            'master',
            'medium',
            'mindblowing',
            'mirror',
            'monocultural',
            'monoculture',
            'moron',
            'motherfucker',
            'motherfuckers',
            'motherfucking',
            'multicultural',
            'nazi',
            'nazis',
            'neonazi',
            'omg',
            'overblown',
            'overlord',
            'oxymoron',
            'people',
            'pervert',
            'perverter',
            'piss',
            'politician',
            'poop',
            'prick',
            'pyramid',
            'racist',
            'redpill',
            'republican',
            'retard',
            'rich',
            'right',
            'sapiosexual',
            'scum',
            'scumbag',
            'scummy',
            'seriously',
            'sexist',
            'shit',
            'shite',
            'shithole',
            'shitload',
            'shitshow',
            'shitstorm',
            'shitton',
            'shitty',
            'sjw', 
            'sleeze',
            'slimeballtype',
            'slimed',
            'slow',
            'skinnysexual',
            'smartarse',
            'socialist',
            'stupid',
            'stupidity',
            'stupidly',
            'suck',
            'sucker',
            'suckiness',
            'superrich',
            'terrorist',
            'trash',
            'troll',
            'trump',
            'tumblrsjw',
            'turd',
            'ugly',
            'undeservedly',
            'unreasonable',
            'vaxx',
            'vaxxed',
            'wank',
            'warrior',
            'waste', 
            'wasteful', 
            'wasteland',
            'weak',
            'weakly',
            'weakness',
            'web',
            'whistleblowing',
            'whore',
            'woman',
            'worthless',
            'wtf',
            'you',
            'your',
            'zomg']
len(wordlist)

220

In [19]:
dump(wordlist, 'wordlist.joblib')

['wordlist.joblib']

In [20]:
dtm_train = dtm_train[wordlist]
dtm_test = dtm_test[wordlist]

In [21]:
y_train_binary.value_counts(normalize=True)

0    0.989594
1    0.010406
Name: papi_toxicity, dtype: float64

In [22]:
model = keras.Sequential()

In [23]:
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]

In [24]:
model.add(keras.layers.Dense(1, input_dim=220, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam')

In [26]:
model.fit(dtm_train, y_train_binary, epochs=25, 
          class_weight = {0: 0.010406, 1: 0.989594},
          callbacks=callbacks,
          validation_split=0.2, 
          verbose=1)

Train on 6380 samples, validate on 1596 samples
Epoch 1/25
6380/6380 [==============================] - ETA: 24:15 - loss: 0.00 - ETA: 12:15 - loss: 0.00 - ETA: 35s - loss: 0.0120 - ETA: 9s - loss: 0.0133 - ETA: 4s - loss: 0.013 - ETA: 3s - loss: 0.013 - ETA: 2s - loss: 0.014 - ETA: 1s - loss: 0.013 - 8s 1ms/sample - loss: 0.0142 - val_loss: 0.0143
Epoch 2/25
6380/6380 [==============================] - ETA: 0s - loss: 0.007 - ETA: 0s - loss: 0.013 - ETA: 0s - loss: 0.014 - ETA: 0s - loss: 0.014 - ETA: 0s - loss: 0.014 - ETA: 0s - loss: 0.014 - 0s 54us/sample - loss: 0.0141 - val_loss: 0.0143
Epoch 3/25
6380/6380 [==============================] - ETA: 0s - loss: 0.028 - ETA: 0s - loss: 0.016 - ETA: 0s - loss: 0.013 - ETA: 0s - loss: 0.014 - ETA: 0s - loss: 0.014 - ETA: 0s - loss: 0.014 - ETA: 0s - loss: 0.014 - 0s 63us/sample - loss: 0.0141 - val_loss: 0.0142
Epoch 4/25
6380/6380 [==============================] - ETA: 0s - loss: 0.048 - ETA: 0s - loss: 0.014 - ETA: 0s - loss: 0.012 -

In [27]:
nn_train_pred = model.predict(dtm_train)
nn_test_pred = model.predict(dtm_test)
pd.DataFrame(nn_train_pred).describe()

,0
count,7976.000000
mean,0.440679
std,0.043458
min,0.414448
25%,0.414861
50%,0.414861
75%,0.452983
max,0.877928


In [28]:
# Calculate metrics on training dataset.
train_accuracy = accuracy_score(y_train_binary, np.rint(nn_train_pred))
train_recall = recall_score(y_train_binary, np.rint(nn_train_pred))
train_precision = precision_score(y_train_binary, np.rint(nn_train_pred))

# Calculate metrics on test dataset.
test_accuracy = accuracy_score(y_test_binary, np.rint(nn_test_pred))
test_recall = recall_score(y_test_binary, np.rint(nn_test_pred))
test_precision = precision_score(y_test_binary, np.rint(nn_test_pred))

In [29]:
# Display metrics for training dataset.
print(f'Train accuracy: {train_accuracy:.4f}')
print(f'Train recall: {train_recall:.4f}')
print(f'Train precision: {train_precision:.4f}', '\n')

# Display metrics for test dataset.
print(f'Test accuracy: {test_accuracy:.4f}')
print(f'Test recall: {test_recall:.4f}')
print(f'Test precision: {test_precision:.4f}')

Train accuracy: 0.9353
Train recall: 0.8193
Train precision: 0.1195 

Test accuracy: 0.9173
Test recall: 0.6000
Test precision: 0.0710


In [30]:
# Calculate test metrics for majority classifier (perdicts no toxic comments).
mc_test_accuracy = accuracy_score(y_test_binary, np.full_like(y_test, 0))
mc_test_recall = recall_score(y_test_binary, np.full_like(y_test, 0))
mc_test_precision = precision_score(y_test_binary, np.full_like(y_test, 0))

C:\Users\aloha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [31]:
# Display metrics for test dataset.
print(f'Majority classifier test accuracy: {mc_test_accuracy:.4f}')
print(f'Majority classifier test recall: {mc_test_recall:.4f}')
print(f'Majority classifier test precision: {mc_test_precision:.4f} (undefined)')

Majority classifier test accuracy: 0.9900
Majority classifier test recall: 0.0000
Majority classifier test precision: 0.0000 (undefined)


In [32]:
# Display comments identified as toxic by custom neural network.
for text in X_test[np.rint(nn_test_pred).squeeze().astype(bool)]:
    print(text, '\n')

Have you considered the possibility that your cow-orkers hate it, and by extension, you? 

Joel didn't say that. You're probably thinking of this piece by Steve Yegge: http://steve-yegge.blogspot.com/2007/02/next-big-language.ht... But I wouldn't exactly characterize that as advice Steve Yegge wants people to follow, either. > Back when I was in the Navy, just out of boot camp, an otherwise entirely forgettable petty officer first class instructor of ours offered us, unasked and free of charge, his sage advice on how to pick up women at a bar: "Go ugly early." With this declaration, he had made it clear that he and I thought rather differently about certain things in life. > If you want to spare yourself a lot of angst in deciding which programming language to use, then I recommend this simple rule: Go ugly early. C++ will go out with you in a heartbeat. > For my part, I want to encourage people to make their own languages 

Yeah because iGoogle had a loyal and dedicated following of u

HusbandWatcher - provide a photo of your cheating husband, and we will scan hotel, car and street surveillance to show you who he is with every moment he is out of your sight Employee-motivator - using simple webcams we can monitor the facial emotions of your employees moment to moment. Are they reacting positively to your email about the new executive jet purchase? Are they showing signs of morale fatigue as they fall behind their schedule? We can send them morale boosting emails such as ... "Smile, you still have your job". 

The author mentions working on Safari. Perhaps writing code to deal with other people's crap has something to do with her views. 

In my case this is what matters most: - Watch what you DRINK, not so much what you eat. Drink water or black coffee/cold-brew or tea most of the time, and turn other beverages into rare treats. Soda is perfectly fine once a week but buy the smallest size and sip it without a straw so that you enjoy it a lot more. (I was shocked at ho

Just got this email: Hi sep332! I made a new product, Zube https://zube.io . It's a simple project management tool for GitHub issues. I've been working on it with my cofounder for 6 months. I'm super excited to announce that we're launching Zube today on Hacker News. You can read more about why we made it here - https://zube.io/blog . Hope you check it out and let me know what you think! Cheers, Jen You know this kind of spam is literally illegal, right? 

He has lots of links. It's the web as TBL intended. 

What does "trolling PTAB" mean? 

It is controversial for those people who hang their hat on the concept that America was in some sort of race to build and use the bomb before the enemy used it against America. No serious historian thinks that was ever a factor once the logistics were apparent, but the History channel still pumps out constant "documentaries" about Nazi superweapons about be deployed had the war lasted another week or two. 

TL;DR: There are infinitely many ways of

Yes. The literally reason LibreSSL uses ComicSans is to troll the "HN Crowd" according to their early homepage. It started as, "help us reach this donation goal to remove comicsans". By this point its just to troll people. Since using comic sans will trigger at least 1 discussion about the font itself. This is actually a good filter community wise. If you won't suffer certain fonts your certainly not gonna stomach hand reading OpenSSL code to figure out its indented goal. 

So? If the payment is less than $10 or less than $50, who cares? The best you can do is warn other people and that sends a stronger message than just getting your money back. Reputation is what counts and you can see this on Amazon and ebay where a poor rep means no future earnings. Good rep means higher potential future earnings. IF someone rips you off you have multiple forums in which to vent your frustration and lower the seller's potential future earnings. That's a lot of power. That's more power than most corp

Jacob, you are conflating a couple of things in my comment. So there is "RPM" for a web site and there is RPM for a web page . No doubt you run analytics on your site to track who goes to which pages and which pages generate the most advertising revenue. In my experience the "landing" page gets a lot of traffic, the "spokes" pages (typically branches to a bunch of related content) come next, and then there are the articles, some of which get a lot of traffic and some which get only a few page views a day/week. You also probably track dwell time so that you can recognize people who land on the page, probably from a Search engine result, and then "bounce" off because it wasn't what they wanted. Or stay and read. Or the unicorn of the bunch, stay, read, and then read a bunch of other pages. I've also been an author in both the print and web world, my columns on JavaWorld continue to get page views and I continue to get various folks popping over to my poorly maintained web site to grab a 

...

BTW: if this concerns the youtranfer.io website (instead of the YouTransfer application), I'm actually a bit hesitant to change this. I'm currently using the GitHub site generator for convenience. There is a limited set of templates available, most of which are either ugly or have readability issues. As the website basically only consists of the README file, you can also look at the GitHub project for more information ( https://github.com/remie/YouTransfer ) 

I worked in a place where I'd be scolded once in a while when I did something stupid and I enjoyed it[0] for a while as long as it was fair. It quickly became annoying when it turned out the same people who criticised others harshly swept their own mistakes under the rug ... [0]: "As iron sharpens iron, so one person sharpens another." 

I'm not disagreeing with you but just feeling like I could look at (A) as be helpful too lots of people and (B) live a selfish life of hedonism. Was that your intent or do you see a different way

I'm not sure what people find "beautiful" or "touching" in this story. A man is wracked by guilt over what he has done to the point of deciding to end his life, but then he forgets all about it as soon as a pretty girl gives him attention. Inasmuch as it is believable, this is a story about how all our lofty notions of justice, honor, purpose etc. are just bullshit, and it's really all about fucking, consuming and keeping those genes alive. Matter over mind, humanity revealed as a mere dusting on thought over the throbbing mass of limbic functions. It is depressing. Its only redeeming quality is the fact that it's obviously fake. 

> IBM Airline Control Program, or ACP, was an operating system developed by IBM beginning about 1965. > TPF evolved from the Airlines Control Program (ACP), a free package developed in the mid-1960s by IBM in association with major North American and European airlines. In 1979, IBM introduced TPF as a replacement for ACP — and as a priced software product. H

Narrow in a sense that it's one specific angle to view math problems. But the range of problems is indeed great; hell, parts of computer science have or may have direct relation to how we view reality itself. 

> Nobody in their right mind wants to live in Russia. There Red Scare is alive and well on the internet, where Russia and China can do no right. 

Were you able to import your Evernote notes? 

It's possible to use Facebook in a way that couldn't endanger your life or career, depending on who you are. Obviously if someone is stalking you or something, that's a different story. But most people can just omit the information that they find to be sensitive and limit their friends list to people they trust. 

Also, what are the levels of "path dependence" that you're referring to? A process model which makes checkpointing and migration difficult, hierarchical file systems, lack of OS-level typed modules, strict kernel-userspace segmentation, the various flavors of user and group IDs 

There's a lot of things that can track presence around the web and gravatar's not one of them. Iff you decide to implement gravatar without mirroring their images, avatars are indeed loaded on their first query (and not subsequent ones) and, god knows what they're doing with that information ohgod . They certainly can't "track my presence around the web", though - no js means no fingerprinting, no tracking cookie, nothing. Juuuust a blind IP address. But the recommended way is to prefetch the avatars directly from your server and offer them on your own cdn. As for the value it provides, well for one thing I pretty much never have to upload my avatar to websites anymore - it's an avatar attached to my email addresses instead and that's very nice. Of course I'd prefer a proper identity protocol but nobody's working on one. If you want to, be my guest... 

> I certainly wouldn't argue that the web should be real-name only 100% agreed. The vast majority of the web is based around pseudonym

@Curiousjorge.. Never, Never quit - Winston Churchill. And he also said when you are going through hell, you keep going. 

Telling people how they should feel is not very nice either. 

If England hadn't kicked off the industrial revolution it might have taken another few centuries to start and you'd be shitting in a hole in the ground in the dark right now. So be grateful for the huge gift they gave all of us. 

Depends on context - over here (north Europe), diesels may be more a symbol of wealth than a gasoline engine. Gas guzzlers are prole, for people who have furry dice in the rear view mirror and silly "vocational school mustache". Compare that to your statement about "smaller petrol engines" for which your example is Macan Turbo 3.6L engine. Over here, such an engine is ridiculously large and almost stupid. A small, efficient petrol engine over here is something like a 1.0 litre turbocharged gas engine in a Ford Mondeo. 

Russian's have been doing this for years because fraudule

In [33]:
model.save('simple_nn.h5')